<a href="https://colab.research.google.com/github/ahmedhesham47/Bayesian-Network-for-Predicting-ICB-Response/blob/main/Bayes_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgmpy

# **Importing Packages**

In [2]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch, BicScore, PC, ExhaustiveSearch, BDsScore, K2Score, BayesianEstimator, MaximumLikelihoodEstimator, MmhcEstimator, BDeuScore
from pgmpy.inference import VariableElimination, BeliefPropagation, CausalInference, Mplp
import random
import matplotlib.pyplot as plt
import networkx as nx
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
import numpy as np
from pgmpy.metrics import correlation_score, log_likelihood_score, structure_score
from networkx.algorithms.approximation import clique
from networkx.algorithms.clique import enumerate_all_cliques
from networkx import find_cliques
from sklearn.model_selection import train_test_split, KFold

# **Loading files**

In [4]:
mrna_cont = pd.read_csv('mrna_6_FS_methods_cont.tsv', sep='\t')
best_e_genes = pd.read_csv('Best Expression Genes_6 combo.tsv', sep='\t')
labels_df = pd.read_csv('labels_df.tsv',sep='\t')
eigen_genes = pd.read_csv('hartmink_cat_eigengenes.tsv', sep='\t')

# clinical = pd.read_csv('processed_clinical.tsv', sep='\t')
# cibersort = pd.read_csv('CIBERSORTx.csv', sep=',')
# cibersort = cibersort.iloc[:, :-4]
# cibersort = cibersort.rename(columns={'Mixture': 'Sample Identifier'})

In [5]:
def split(df, test_size=0.2):
  train_df, test_df = train_test_split(df, test_size=test_size, random_state=42)
  return train_df, test_df

# **Discretizing the dataframe**

## **General Std. Categorization Function**

In [6]:
def categorize_columns(df, columns, units_std=1):
    for col in columns:
        if col in df.columns:
            mean = df[col].mean()
            std = df[col].std()

            # Apply transformation
            df[col] = df[col].apply(lambda x: 1 if x > mean + (units_std*std) else (-1 if x < mean - (units_std*std) else 0))
        else:
            print(f"Column '{col}' not found in DataFrame.")
    return df

In [106]:
to_be_categorical = ['LDH (treatment Start)', 'Total Mutations',	'Nonsynonymous Mutation Count',	'Mutations Clonal',	'Mutations Subclonal', 'Heterogeneity',	'Total Neoantigen',
'CNA Prop', 'Purity', 'Ploidy', 'UV-Induced Mutations', 'Alkylating Chemotherapy']

# clinical = categorize_columns(clinical, to_be_categorical)

# cibersort = categorize_columns(cibersort, cibersort.columns[1:], 0.25)

# model_df = pd.merge(mrna_categorized, clinical, on='Sample Identifier')
# model_df = pd.merge(model_df, cibersort, on='Sample Identifier')

# column_to_move = model_df['ICB Response']
# model_df = model_df.drop('ICB Response', axis=1)
# model_df['ICB Response'] = column_to_move

mrna_cont = categorize_columns(mrna_cont, mrna_cont.iloc[:, 1:].columns)
mrna_categorized_labelled = pd.merge(mrna_cont, labels_df, on='Sample Identifier')
mrna_categorized_labelled['ICB Response'] = mrna_categorized_labelled['ICB Response'].astype(int)
mrna_categorized_labelled.to_csv('mrna_categorized_labelled_filtered_stringent.tsv', sep='\t', index=False)

# **Bayesian Network (genes arising from FS methods)**

In [ ]:
train_df = split(mrna_categorized_labelled)[0]
test_df = split(mrna_categorized_labelled)[1]

edges_to_exclude = []
for col in mrna_categorized_labelled.iloc[:, 1:].columns:
  edges_to_exclude.append(('ICB Response', col))

bic_score = BicScore(mrna_categorized_labelled.iloc[:,1:])
K2C_score = K2Score(mrna_categorized_labelled.iloc[:,1:])
BDs_score = BDsScore(mrna_categorized_labelled.iloc[:,1:])
BDeu_score = BDeuScore(mrna_categorized_labelled.iloc[:,1:])

hc = HillClimbSearch(mrna_categorized_labelled.iloc[:,1:])
# mhmc = MmhcEstimator(mrna_categorized_labelled.iloc[:,1:])
# pc = PC(mrna_categorized_labelled.iloc[:,1:])

best_model = hc.estimate(scoring_method=bic_score, max_iter=1000, black_list=edges_to_exclude)
best_model2 = hc.estimate(scoring_method=K2C_score, max_iter=20, black_list=edges_to_exclude)
best_model3 = hc.estimate(scoring_method=BDs_score, max_iter=1000, black_list=edges_to_exclude)
best_model4 = hc.estimate(scoring_method=BDeu_score, max_iter=1000, black_list=edges_to_exclude)

# best_model = mhmc.estimate(scoring_method=bic_score, significance_level=0.05)
# best_model2 = mhmc.estimate(scoring_method=K2C_score, significance_level=0.05)
# best_model3 = mhmc.estimate(scoring_method=BDs_score, significance_level=0.05)
# best_model4 = mhmc.estimate(scoring_method=BDeu_score, significance_level=0.05)

# best_model = pc.estimate(ci_test = 'chi_square', significance_level=0.05)
# best_model2 = pc.estimate(ci_test = 'g_sq', significance_level=0.05)
# best_model3 = pc.estimate(ci_test = 'log_likelihood', significance_level=0.05)
# best_model4 = pc.estimate(ci_test = 'cressie_read', significance_level=0.05)

model = BayesianNetwork(best_model.edges())
model2 = BayesianNetwork(best_model2.edges())
model3 = BayesianNetwork(best_model3.edges())
model4 = BayesianNetwork(best_model4.edges())

model.fit(train_df.iloc[:, 1:], estimator=MaximumLikelihoodEstimator)
model2.fit(train_df.iloc[:, 1:], estimator=MaximumLikelihoodEstimator)
model3.fit(train_df.iloc[:, 1:], estimator=MaximumLikelihoodEstimator)
model4.fit(train_df.iloc[:, 1:], estimator=MaximumLikelihoodEstimator)

In [100]:
models = {'BiC': model, 'K2S': model2, 'BDs': model3, 'BDe': model4}
icb_response_edges = {name: [] for name in models}

for name, model in models.items():
    for edge in model.edges:
        if edge[1] == 'ICB Response':
            icb_response_edges[name].append(edge)

print(icb_response_edges)

{'BiC': [('ATPIF1', 'ICB Response'), ('TMEM8A', 'ICB Response'), ('XRCC3', 'ICB Response'), ('ZNF462', 'ICB Response')], 'K2S': [('ATPIF1', 'ICB Response')], 'BDs': [('ATPIF1', 'ICB Response'), ('KIAA0391', 'ICB Response')], 'BDe': [('ATPIF1', 'ICB Response'), ('TMEM8A', 'ICB Response'), ('XRCC3', 'ICB Response'), ('ZNF462', 'ICB Response')]}


# **Bayesian Network (eigen-genes Hartemink Discretization)**

In [134]:
# cols_to_keep = ['Sample Identifier','MEgrey', 'MEtan', 'MEpurple', 'MEskyblue', 'MEpink', 'MEgreenyellow', 'MEgreen', 'MEsaddlebrown', 'MElightgreen', 'ICB Response']
# eigen_genes = eigen_genes[cols_to_keep]

train_df = split(eigen_genes)[0]
test_df = split(eigen_genes)[1]

edges_to_exclude = []
for col in eigen_genes.iloc[:, 1:].columns:
  edges_to_exclude.append(('ICB Response', col))

bic_score = BicScore(eigen_genes.iloc[:,1:])
K2C_score = K2Score(eigen_genes.iloc[:,1:])
BDs_score = BDsScore(eigen_genes.iloc[:,1:])
BDeu_score = BDeuScore(eigen_genes.iloc[:,1:])

hc = HillClimbSearch(eigen_genes.iloc[:,1:])
# mhmc = MmhcEstimator(eigen_genes.iloc[:,1:])
# pc = PC(eigen_genes.iloc[:,1:])

best_model = hc.estimate(scoring_method=bic_score, max_iter=1000, black_list=edges_to_exclude)
best_model2 = hc.estimate(scoring_method=K2C_score, max_iter=30, black_list=edges_to_exclude)
best_model3 = hc.estimate(scoring_method=BDs_score, max_iter=1000, black_list=edges_to_exclude)
best_model4 = hc.estimate(scoring_method=BDeu_score, max_iter=1000, black_list=edges_to_exclude)

# best_model = mhmc.estimate(scoring_method=bic_score, significance_level=0.05)
# best_model2 = mhmc.estimate(scoring_method=K2C_score, significance_level=0.05)
# best_model3 = mhmc.estimate(scoring_method=BDs_score, significance_level=0.05)
# best_model4 = mhmc.estimate(scoring_method=BDeu_score, significance_level=0.05)

# best_model = pc.estimate(ci_test = 'chi_square', significance_level=0.05)
# best_model2 = pc.estimate(ci_test = 'g_sq', significance_level=0.05)
# best_model3 = pc.estimate(ci_test = 'log_likelihood', significance_level=0.05)
# best_model4 = pc.estimate(ci_test = 'neyman', significance_level=0.05)

model = BayesianNetwork(best_model.edges())
model2 = BayesianNetwork(best_model2.edges())
model3 = BayesianNetwork(best_model3.edges())
model4 = BayesianNetwork(best_model4.edges())

model.fit(train_df.iloc[:, 1:], estimator=BayesianEstimator)
model2.fit(train_df.iloc[:, 1:], estimator=BayesianEstimator)
model3.fit(train_df.iloc[:, 1:], estimator=BayesianEstimator)
model4.fit(train_df.iloc[:, 1:], estimator=BayesianEstimator)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

# **Testing the model**

In [74]:
inference = VariableElimination(model4)

In [75]:
model_nodes = list(model.nodes)
model2_nodes = list(model2.nodes)
model3_nodes = list(model3.nodes)
model4_nodes = list(model4.nodes)

model_data = mrna_categorized_labelled.iloc[:,1:][model_nodes]
model2_data = mrna_categorized_labelled.iloc[:,1:][model2_nodes]
model3_data = mrna_categorized_labelled.iloc[:,1:][model3_nodes]
model4_data = mrna_categorized_labelled.iloc[:,1:][model4_nodes]

In [ ]:
correlation_score(model4, model4_data)
log_likelihood_score(model4, model4_data)
structure_score(model4, model4_data)

In [63]:
samples = test_df[['Sample Identifier', 'ICB Response']]
sample_ids = list(samples['Sample Identifier'])

In [ ]:
list_of_dicts = test_df.iloc[:,1:-1].to_dict(orient='records')
predictions = {}
for sample_dict, s in zip(list_of_dicts, sample_ids):
  filtered_dict = {key: sample_dict[key] for key in model4_nodes if key in sample_dict}
  query_result = inference.map_query(variables=['ICB Response'], evidence=filtered_dict)
  predictions[s] = query_result['ICB Response']

In [ ]:
predictions_df = pd.DataFrame.from_dict(predictions, orient='index', columns=['ICB Response'])
predictions_df.reset_index(inplace=True)
predictions_df.columns = ['Sample Identifier', 'ICB Response']

samples.reset_index(inplace=True)
samples.drop('index', axis=1, inplace=True)

In [78]:
((predictions_df == samples).sum() / len(samples)) * 100

Sample Identifier    100.0
ICB Response          62.5
dtype: float64

# **Network Visualization**

In [ ]:
# Create a NetworkX graph from the Bayesian network
G = nx.DiGraph()
G.add_edges_from(best_model4.edges())

# Draw the graph
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
pos = nx.spring_layout(G)  # Layout for the nodes

# Draw nodes and edges
nx.draw(G, pos, with_labels=True, node_size=1500, node_color="lightblue", font_size=5, font_weight="bold", edge_color="gray")

# Display the graph
plt.title("Bayesian Network Visualization")
plt.savefig('Bayes_network.png')
plt.show()

In [104]:
edges_list = [edge for edge in best_model4.edges()]
edges_df = pd.DataFrame(edges_list, columns=['Feature 1', 'Feature 2'])
edges_df.to_csv('eigen_gene_interactions_bde_6_FS_methods.tsv', sep='\t', index=False)

# **Quantifying the strength of edges**

In [ ]:
edge_data = []

for parent, child in best_model.edges:
    cpt = model.get_cpds(node=child)

    influence_measure = cpt.values.ptp()  #ptp = peak-to-peak (max-min) of the probabilities

    edge_data.append((parent, child, influence_measure))

edges_df = pd.DataFrame(edge_data, columns=['Gene 1', 'Gene 2', 'Association Strength'])

edges_df.to_csv('gene_interactions_strength_quantified.tsv', sep='\t', index=False)